In [1]:
import re
from pathlib import Path

import numpy as np
from scipy import optimize

In [2]:
input_data = Path("example.txt").read_text()
print(input_data)

[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}



In [3]:
pattern = re.compile(
    r"\[(?P<light>[.#]+)\] "
    r"(?P<buttons>(\(\d+(,\d+)*\) )+)"
    r"\{(?P<joltage>\d+(,\d+)*)\}",
)
lines = input_data.strip().splitlines()

lights = []
buttons = []
joltages = []
for line in lines:
    match = pattern.match(line)

    light_str = match.group("light").strip()
    lights.append(np.array([c == "#" for c in light_str], dtype=int))

    buttons_str = match.group("buttons").strip()
    buttons.append(
        [
            np.array(button[1:-1].split(","), dtype=int)
            for button in buttons_str.split(" ")
        ],
    )

    joltage_str = match.group("joltage").strip()
    joltages.append(np.array(joltage_str.split(","), dtype=int))


## Part I

In [4]:
def generate_binary_combinations(num_bits: int) -> np.ndarray:
    num_combinations = 2**num_bits
    binary_combinations_list = [
        # create list of '0' and '1' characters
        # String representation appears to be faster than bitwise
        list(np.binary_repr(i, width=num_bits))
        for i in range(num_combinations)
    ]
    binary_combinations = np.array(binary_combinations_list, dtype=int)
    # sort by number of bits set (buttons pressed)
    bit_counts = binary_combinations.sum(axis=1)
    sorted_indices = np.argsort(bit_counts)
    return binary_combinations[sorted_indices]


total_buttons_pressed = 0
for target_lights, button_indices in zip(lights, buttons, strict=True):
    num_target_lights = len(target_lights)
    num_buttons = len(button_indices)

    buttons_pressed = np.zeros((num_buttons, num_target_lights), dtype=int)
    for i, index in enumerate(button_indices):
        buttons_pressed[i, index] = True

    button_combinations = generate_binary_combinations(num_buttons)
    lights_on = (button_combinations @ buttons_pressed) % 2
    candidates = np.all(lights_on == target_lights, axis=1)
    fewest_buttons_index = np.argwhere(candidates).min()
    fewest_buttons_pressed = button_combinations[fewest_buttons_index].sum()
    total_buttons_pressed += fewest_buttons_pressed

print(total_buttons_pressed)


7


## Part II

In [5]:
solutions = []
for button_indices, target_joltage in zip(buttons, joltages, strict=True):
    num_target_joltage = len(target_joltage)

    num_buttons = len(button_indices)

    buttons_pressed = np.zeros((num_buttons, num_target_joltage), dtype=int)
    for i, index in enumerate(button_indices):
        buttons_pressed[i, index] = True

    solution = (
        optimize.linprog(
            c=np.ones(num_buttons, dtype=int),
            A_eq=buttons_pressed.T,
            b_eq=target_joltage,
            integrality=1,
        )
        .x.round()
        .astype(int)
    )
    solutions.append(solution.sum())

print(sum(solutions))

33
